In [11]:
# !wget https://gist.githubusercontent.com/huseinzol05/98974ae8c6c7a65d4bc0af9f5003786a/raw/2e06e71ef7349a57bc58cc9913ae6bae1f9f8447/mp.py
# !wget https://huggingface.co/datasets/mesolitica/instructions-dataset/resolve/main/shuf-combine-malay-no-alignment-multitasks-v5.jsonl

In [5]:
# !split -l 213000 -d --additional-suffix=.splitted shuf-combine-malay-no-alignment-multitasks-v5.jsonl shuf-combine-malay-no-alignment-multitasks-v5.jsonl

In [6]:
!ls shuf-combine-malay-no-alignment-multitasks-v5.jsonl*

shuf-combine-malay-no-alignment-multitasks-v5.jsonl
shuf-combine-malay-no-alignment-multitasks-v5.jsonl00.splitted
shuf-combine-malay-no-alignment-multitasks-v5.jsonl01.splitted
shuf-combine-malay-no-alignment-multitasks-v5.jsonl02.splitted
shuf-combine-malay-no-alignment-multitasks-v5.jsonl03.splitted
shuf-combine-malay-no-alignment-multitasks-v5.jsonl04.splitted
shuf-combine-malay-no-alignment-multitasks-v5.jsonl05.splitted
shuf-combine-malay-no-alignment-multitasks-v5.jsonl06.splitted
shuf-combine-malay-no-alignment-multitasks-v5.jsonl07.splitted
shuf-combine-malay-no-alignment-multitasks-v5.jsonl08.splitted
shuf-combine-malay-no-alignment-multitasks-v5.jsonl09.splitted
shuf-combine-malay-no-alignment-multitasks-v5.jsonl10.splitted
shuf-combine-malay-no-alignment-multitasks-v5.jsonl11.splitted
shuf-combine-malay-no-alignment-multitasks-v5.jsonl12.splitted
shuf-combine-malay-no-alignment-multitasks-v5.jsonl13.splitted
shuf-combine-malay-no-alignment-multitasks-v5.jsonl14.splitted
shu

In [14]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('mesolitica/mistral-7b-4096-fpf')
tokenizer.pad_token = tokenizer.unk_token
tokenizer.add_bos_token = False
tokenizer.add_eos_token = False
tokenizer.padding_side = "right"
tokenizer.chat_template = "{{ bos_token }}{% for message in messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}{{ raise_exception('Conversation roles must alternate user/assistant/user/assistant/...') }}{% endif %}{% if message['role'] == 'user' %}{{ '[INST] ' + message['content'] + ' [/INST]' }}{% elif message['role'] == 'assistant' %}{{ message['content'] + eos_token}}{% else %}{{ raise_exception('Only user and assistant roles are supported!') }}{% endif %}{% endfor %}"

In [8]:
def generate_and_tokenize_prompt(row):
    if '<bot>:' in row['input'] and row['output'] is None:
        inputs, outputs = [], []
        splitted = row['input'].split('<bot>:')
        for i in range(len(splitted) - 1):
            if i == 0:
                human = splitted[i].replace('<manusia>:', '')
            else:
                try:
                    human = splitted[i].split('<manusia>:')[1]
                except BaseException:
                    continue
            bot = splitted[i + 1].split('<manusia>:')[0]
            inputs.append(human)
            outputs.append(bot)
    else:
        inputs = [row['input']]
        outputs = [row['output']]

    chat = []
    for input, output in zip(inputs, outputs):
        chat.extend([
            {'role': 'user', 'content': input.strip()},
            {'role': 'assistant', 'content': output.strip()},
        ])
    prompt = tokenizer.apply_chat_template(chat, tokenize=False)
    return {'text': prompt}

In [31]:
from streaming import MDSWriter
from streaming.base.format.mds.encodings import Encoding, _encodings
from streaming import LocalDataset
import streaming
import numpy as np
from tqdm import tqdm
from glob import glob
import os
import json

class UInt16(Encoding):
    def encode(self, obj) -> bytes:
        return obj.tobytes()

    def decode(self, data: bytes):
        return np.frombuffer(data, np.uint16)

_encodings['uint16'] = UInt16

columns = {
    'input_ids': 'uint16',
}
hashes = 'sha1', 'xxh64'

In [32]:
!mkdir tokenized_mistral

mkdir: cannot create directory ‘tokenized_mistral’: File exists


In [56]:
def loop(files, block_size = 16384):
    files, index = files
    out_root = f'tokenized_mistral/tokenized-{index}'
    os.system(f'rm -rf {out_root}')
    temp = []
    with MDSWriter(out=out_root, columns=columns, compression=None, hashes=hashes) as out:
        for f in files:
            with open(f) as fopen:
                for l in tqdm(fopen):
                    row = json.loads(l)
                    element = generate_and_tokenize_prompt(row)
                    outputs = tokenizer(element['text'])
                    temp.extend(outputs['input_ids'])
                    done = False
                    while len(temp) >= block_size:
                        block = temp[:block_size]
                        temp = temp[block_size:]
                        if len(block) == block_size:
                            out.write({
                                'input_ids': np.array(block).astype(np.uint16)
                            })

In [57]:
files = sorted(glob('shuf-combine-malay-no-alignment-multitasks-v5.jsonl*.splitted'), key = lambda x: int(x.split('jsonl')[-1].split('.')[0]))
files

['shuf-combine-malay-no-alignment-multitasks-v5.jsonl00.splitted',
 'shuf-combine-malay-no-alignment-multitasks-v5.jsonl01.splitted',
 'shuf-combine-malay-no-alignment-multitasks-v5.jsonl02.splitted',
 'shuf-combine-malay-no-alignment-multitasks-v5.jsonl03.splitted',
 'shuf-combine-malay-no-alignment-multitasks-v5.jsonl04.splitted',
 'shuf-combine-malay-no-alignment-multitasks-v5.jsonl05.splitted',
 'shuf-combine-malay-no-alignment-multitasks-v5.jsonl06.splitted',
 'shuf-combine-malay-no-alignment-multitasks-v5.jsonl07.splitted',
 'shuf-combine-malay-no-alignment-multitasks-v5.jsonl08.splitted',
 'shuf-combine-malay-no-alignment-multitasks-v5.jsonl09.splitted',
 'shuf-combine-malay-no-alignment-multitasks-v5.jsonl10.splitted',
 'shuf-combine-malay-no-alignment-multitasks-v5.jsonl11.splitted',
 'shuf-combine-malay-no-alignment-multitasks-v5.jsonl12.splitted',
 'shuf-combine-malay-no-alignment-multitasks-v5.jsonl13.splitted',
 'shuf-combine-malay-no-alignment-multitasks-v5.jsonl14.splitt

In [58]:
import mp
mp.multiprocessing(files, loop, cores = min(len(files), 30), returned = False)

196786it [09:21, 350.39it/s]
213000it [09:53, 358.98it/s]
213000it [09:54, 358.49it/s]
213000it [09:54, 358.07it/s]
213000it [09:54, 358.01it/s]
213000it [09:56, 356.96it/s]
213000it [09:57, 356.74it/s]
213000it [09:57, 356.25it/s]
213000it [09:58, 355.92it/s]
213000it [09:59, 355.39it/s]
213000it [10:00, 354.88it/s]
213000it [10:01, 354.31it/s]
213000it [10:02, 353.79it/s]
213000it [10:02, 353.30it/s]
213000it [10:04, 352.53it/s]
213000it [10:07, 350.90it/s]
213000it [10:09, 349.21it/s]
213000it [10:10, 349.08it/s]
213000it [10:11, 348.51it/s]
213000it [10:12, 347.49it/s]


In [60]:
folders = sorted(glob('tokenized_mistral/tokenized-*'), key = lambda x: int(x.split('-')[-1]))
folders

['tokenized_mistral/tokenized-0',
 'tokenized_mistral/tokenized-1',
 'tokenized_mistral/tokenized-2',
 'tokenized_mistral/tokenized-3',
 'tokenized_mistral/tokenized-4',
 'tokenized_mistral/tokenized-5',
 'tokenized_mistral/tokenized-6',
 'tokenized_mistral/tokenized-7',
 'tokenized_mistral/tokenized-8',
 'tokenized_mistral/tokenized-9',
 'tokenized_mistral/tokenized-10',
 'tokenized_mistral/tokenized-11',
 'tokenized_mistral/tokenized-12',
 'tokenized_mistral/tokenized-13',
 'tokenized_mistral/tokenized-14',
 'tokenized_mistral/tokenized-15',
 'tokenized_mistral/tokenized-16',
 'tokenized_mistral/tokenized-17',
 'tokenized_mistral/tokenized-18',
 'tokenized_mistral/tokenized-19']

In [61]:
!rm -rf packing-mistral

In [62]:
with MDSWriter(out='packing-mistral', columns=columns, compression=None, hashes=hashes) as out:
    for f in folders:
        try:
            dataset = LocalDataset(local=f)
            for i in tqdm(range(len(dataset))):
                out.write(dataset[i])
        except Exception as e:
            print(e)
            pass

100%|██████████| 14627/14627 [00:01<00:00, 9693.25it/s]


In [63]:
dataset = LocalDataset('packing-mistral')

In [64]:
len(dataset) * 16384

5152473088

In [66]:
dataset[-2]

{'input_ids': array([1883, 2950, 4292, ..., 1883, 2950, 1626], dtype=uint16)}